In [1]:
## import
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 

## This is for data handling.
# import cv2
# import time
# import math as m
# import MediaPipe as mp


2023-02-02 16:01:23.030359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 16:01:23.723966: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/extras/CUPTI/lib64:/usr/local/cuda-11.2/lib64:
2023-02-02 16:01:23.726141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/extras/CUPTI/lib64:/usr/local/cuda-11.2/l

- To use MediaPipe to make a Convolutional Neural Network (CNN) model for pose prediction, you would follow these steps:

1. Install MediaPipe: To use MediaPipe, you'll first need to install it. You can do this by following the installation instructions on the MediaPipe GitHub repository.

2. Gather data: You'll need a dataset of human poses to train your CNN model. You can either gather your own data or use a publicly available dataset.

3. Preprocess data: Before training the CNN model, you'll need to preprocess the data to ensure that it's in the correct format and has been normalized.

4. Define the CNN model architecture: In this step, you'll need to define the structure of the CNN model, including the number of layers, the type of layers, and the activation functions.

5. Train the CNN model: Using the preprocessed data, you'll train the CNN model using a suitable deep learning framework such as TensorFlow or PyTorch.

6. Evaluate the CNN model: After training the model, you'll need to evaluate its performance on a validation dataset to ensure that it's working as expected.

- Use MediaPipe to run the CNN model: Once the CNN model is trained and working, you can use MediaPipe to integrate it into a processing pipeline. To do this, you'll need to write code that initializes the pipeline, sets up the input and output streams, and starts the processing loop.

- Note: The exact steps to use MediaPipe for pose prediction may vary depending on the specific pipeline you're using and the programming language you're using. I suggest that you refer to the official MediaPipe documentation and other relevant resources for more detailed instructions.

In [ ]:
# Just REFERENCE 1 코드
# 데이터 전처리
# 4개의 파일을 사용
data_stand_woman = np.loadtxt(fileUrl + '/stand/data_stand_woman.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_stand_man = np.loadtxt(fileUrl + '/stand/data_stand_man.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_sit_woman = np.loadtxt(fileUrl + '/sit/data_sit_woman.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_sit_man = np.loadtxt(fileUrl + '/sit/data_sit_man.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)

# 두 명의 서 있는 자세와 앉아 있는 자세 병합
data_stand = np.concatenate((data_stand_woman, data_stand_man), axis = 1).transpose()
data_sit = np.concatenate((data_sit_woman, data_sit_man), axis = 1).transpose()

# 분류를 위한 라벨 구성 (0 - 서 있는 자세, 1 - 앉아 있는 자세)
label_stand = []
label_sit = []
for i in range(len(data_stand)):
  label_stand.append(0)
for i in range(len(data_sit)):
  label_sit.append(1)

# 앉아 있는 자세와 서 있는 자세에 대한 데이터를 합쳐서 데이터 전처리 마무리
data = np.concatenate((data_stand, data_sit), axis = 0)
label = np.array(label_stand + label_sit)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 121)

print(X_train.shape)
print(y_train.shape)

In [ ]:
# first Model 2023/2/02

model = keras.Sequential([
    keras.layers.Dense(10, input_shape = (26,), activation='relu'), # 26개의 관절 좌표
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
#train
model.fit(X_train, y_train, batch_size = 50, epochs = 50, verbose = 1) # train

In [ ]:
# evaluate
results = model.evaluate(X_test, y_test)
print('accuracy: ', results[1])

In [ ]:
# model 2 
# I made it.

import tensorflow as tf

# Architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3))) #BGR or RGB
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax')) # normal, or alert

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your training data
model.fit(X_train, y_train, epochs=10)

In [ ]:
# train 2
model.fit(X_train, y_train, epochs=10)

In [ ]:
# evaluate 2
results = model.evaluate(X_test, y_test)
print('accuracy: ', results[1])

In [ ]:
model.save('Maded_model.h5')

In [ ]:
# model 3, Just Reference 2 code, hand predict mediapipe model
# Reference 2 = https://github.com/Rishav-hub/MediaPipe-Tutorial/blob/main/MediaPipe.ipynb
# It used several ML model for prediction.

pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
} #default

In [ ]:
# Train 3 
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
# Evaluate 3

for algo, model in fit_models.items():
    predicted_y = model.predict(X_test)
    print(algo, accuracy_score(y_test, predicted_y))

In [ ]:
# # real check

# with open('Numbers.pkl', 'rb') as f:
#     model = pickle.load(f)

In [ ]:
# cap = cv2.VideoCapture(0)
# with mp.solutions.pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as mp_pose:
#     while cap.isOpened():
#         success, image = cap.read()
       
#         # Flip the image horizontally for a later selfie-view display, and convert
#         # the BGR image to RGB.
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         # To improve performance, optionally mark the image as not writeable to
#         # pass by reference.
#         image.flags.writeable = False
#         results = mp_pose.process(image)

#         # Draw the hand annotations on the image.
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)



# #         image_height, image_width, _ = image.shape

#         if results.multi_hand_landmarks:
#             for hand_landmarks in results.multi_hand_landmarks:
#                 mp_drawing.draw_landmarks(
#                     image, hand_landmarks, mp_pose.HAND_CONNECTIONS)
#         try:
#             hand = hand_landmarks.landmark
#             hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in hand]).flatten())

#             row = hand_row
# #             print(row)


#             X = pd.DataFrame([row])
#             hand_language_class = model.predict(X)[0]
#             hand_language_prob = model.predict_proba(X)[0]
#             print(hand_language_class, hand_language_prob)



#             coords = tuple(np.multiply(
#                             np.array(
#                                 (hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x, 
#                                  hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y))
#                         , [640,480]).astype(int))

#             cv2.rectangle(image, 
#                           (coords[0], coords[1]+5), 
#                           (coords[0]+len(hand_language_class)*20, coords[1]-30), 
#                           (245, 117, 16), -1)
#             cv2.putText(image, hand_language_class, coords, 
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#             # Get status box
#             cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

#             # Display Class
#             cv2.putText(image, 'CLASS'
#                         , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, hand_language_class.split(' ')[0]
#                         , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#             # Display Probability
#             cv2.putText(image, 'PROB'
#                         , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, str(round(hand_language_prob[np.argmax(hand_language_prob)],2))
#                         , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

#         except:
#             pass
            
   
#         cv2.imshow('MediaPipe Hands 2', image)
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
# cap.release()
# cv2.destroyAllWindows()